## Multiclass relations classification used in tree building

1. prepare train/test sets
2. generate config files for bimpm model
3. generate training/prediction script
4. model evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import pandas as pd
import pickle
from utils.file_reading import read_edus, read_gold, read_negative, read_annotation

### Make a directory

In [ ]:
MODEL_PATH = 'models/label_predictor_lstm'
! mkdir $MODEL_PATH

TRAIN_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_train.tsv')
DEV_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_dev.tsv')
TEST_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_test.tsv')

### 1. prepare train/test sets 

In [ ]:
from utils.train_test_split import split_train_dev_test

train, dev, test = split_train_dev_test('./data')

In [ ]:
MAX_LEN = 250

In [ ]:
from tqdm.autonotebook import tqdm

TARGET = 'category_id'
random_state = 45
train_samples = []

for file in tqdm(train):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order', 'filename']]
    train_samples.append(sample)

train_samples = pd.concat(train_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
train_samples.reset_index(level=0, inplace=True)
train_samples[TARGET] = train_samples[TARGET].replace([0.0], 'same-unit_m')
train_samples['order'] = train_samples['order'].replace([0.0], 'NN')
train_samples[TARGET] = train_samples[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
train_samples[TARGET] = train_samples[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')
train_samples[TARGET] = train_samples[TARGET].replace(['motivation_r',], 'condition_r')

In [ ]:
train_samples

In [ ]:
train_samples[TARGET].value_counts()

In [ ]:
train_samples['relation'] = train_samples[TARGET].map(lambda row: row[:-1]) + train_samples['order']
train_samples['relation'].value_counts()

In [ ]:
train_samples['relation'] = train_samples['relation'].replace(['solutionhood_NS', 'antithesis_NS', 'preparation_NS'], 'elaboration_NS')
train_samples['relation'] = train_samples['relation'].replace(['antithesis_SN', 'preparation_NS',
                                                               'concession_SN', 'evaluation_SN', 
                                                               'elaboration_SN', 'evidence_SN'], 'preparation_SN')
train_samples['relation'].value_counts()

In [ ]:
NUMBER_CLASSES = len(train_samples['relation'].unique())
NUMBER_CLASSES

In [ ]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].head()

In [ ]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TRAIN_FILE_PATH, sep='\t', header=False, index=False)

#### Dev/test sets

In [ ]:
random_state = 45
dev_samples = []

for file in tqdm(dev):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order']]
    dev_samples.append(sample)

dev_samples = pd.concat(dev_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
dev_samples.reset_index(level=0, inplace=True)
dev_samples[TARGET] = dev_samples[TARGET].replace([0.0], 'same-unit_m')
dev_samples['order'] = dev_samples['order'].replace([0.0], 'NN')
dev_samples[TARGET] = dev_samples[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['motivation_r',], 'condition_r')

In [ ]:
dev_samples['relation'] = dev_samples['relation'].replace(['solutionhood_NS', 'antithesis_NS', 'preparation_NS'], 'elaboration_NS')
dev_samples['relation'] = dev_samples['relation'].replace(['antithesis_SN', 'preparation_NS',
                                                           'concession_SN', 'evaluation_SN', 
                                                           'elaboration_SN', 'evidence_SN'], 'preparation_SN')
dev_samples['relation'].value_counts()

In [ ]:
dev_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(DEV_FILE_PATH, sep='\t', header=False, index=False)

In [ ]:
len(dev_samples['relation'].unique())

In [ ]:
random_state = 45
test_samples = []

for file in tqdm(test):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order', 'filename']]
    test_samples.append(sample)

test_samples = pd.concat(test_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
test_samples.reset_index(level=0, inplace=True)
test_samples[TARGET] = test_samples[TARGET].replace([0.0], 'same-unit_m')
test_samples['order'] = test_samples['order'].replace([0.0], 'NN')
test_samples[TARGET] = test_samples[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
test_samples[TARGET] = test_samples[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')
test_samples[TARGET] = test_samples[TARGET].replace(['motivation_r',], 'condition_r')
test_samples['relation'] = test_samples[TARGET].map(lambda row: row[:-1]) + test_samples['order']
test_samples['relation'] = test_samples['relation'].replace(['solutionhood_NS', 'antithesis_NS', 'preparation_NS'], 'elaboration_NS')
test_samples['relation'] = test_samples['relation'].replace(['antithesis_SN', 'preparation_NS',
                                                           'concession_SN', 'evaluation_SN', 
                                                           'elaboration_SN', 'evidence_SN'], 'preparation_SN')

test_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TEST_FILE_PATH, sep='\t', header=False, index=False)

In [ ]:
test_samples.relation.value_counts()

### 2. Generate config files

In [ ]:
%%writefile $MODEL_PATH/config_bert.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "token_indexers": {
      "bert": {
          "type": "bert-pretrained",
          "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
          "do_lowercase": false,
          "use_starting_offsets": true
      },
      "token_characters": {
          "type": "characters",
          "min_padding_length": 1
      }
    }
  },
  "train_data_path": "label_predictor_lstm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_lstm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_lstm/nlabel_cf_test.tsv",
  "model": {
    "type": "bimpm",
    "dropout": 0.1,
    "text_field_embedder": {
        "allow_unmatched_keys": true,
        "embedder_to_indexer_map": {
            "bert": ["bert", "bert-offsets"],
            "token_characters": ["token_characters"],
        },
        "token_embedders": {
            "bert": {
                "type": "bert-pretrained",
                "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true
              }
            }
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 768+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 768+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 2,
      "dropout": 0.5
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 2,
      "hidden_dims": [200, 22],
      "activations": ["tanh", "linear"],
      "dropout": [0.5, 0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ]
  },
  "iterator": {
    "type": "basic",
    "batch_size": 2
  },
  "trainer": {
    "num_epochs": 50,
    "patience": 10,
    "cuda_device": 0,
    "grad_norm": 10.0,
    "validation_metric": "+accuracy",
    "optimizer": {
      "type": "bert_adam",
      "lr": 0.002
    }
  }
}

In [ ]:
%%writefile $MODEL_PATH/config_elmo.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "token_indexers": {
      "token_characters": {
        "type": "characters",
        "min_padding_length": 3
      },
      "elmo": {
        "type": "elmo_characters"
     }
    }
  },
  "train_data_path": "label_predictor_lstm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_lstm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_lstm/nlabel_cf_test.tsv",
  "model": {
    "type": "bimpm",
    "dropout": 0.1,
    "text_field_embedder": {
        "token_embedders": {
            "elmo": {
                    "type": "elmo_token_embedder",
                    "options_file": "rsv_elmo/options.json",
                    "weight_file": "rsv_elmo/model.hdf5",
                    "do_layer_norm": false,
                    "dropout": 0.0
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true
              }
            }
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 1024+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 1024+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 2,
      "dropout": 0.5
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 2,
      "hidden_dims": [200, 22],
      "activations": ["tanh", "linear"],
      "dropout": [0.5, 0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ]
  },
  "iterator": {
    "type": "basic",
    "batch_size": 2
  },
  "trainer": {
    "num_epochs": 50,
    "patience": 10,
    "cuda_device": 0,
    "grad_norm": 10.0,
    "validation_metric": "+accuracy",
    "optimizer": {
      "type": "adam",
      "lr": 0.001
    }
  }
}

### 3. Script for training/prediction 

In [ ]:
%%writefile models/train_label_predictor.sh
# usage:
# $ cd models 
# $ sh train_label_predictor.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="nlabel_cf_dev.tsv"
export TEST_FILE_PATH="nlabel_cf_test.tsv"

rm -r label_predictor_lstm/${RESULT_DIR}/
allennlp train -s label_predictor_lstm/${RESULT_DIR}/ label_predictor_lstm/config_${METHOD}.json
allennlp predict --use-dataset-reader --silent --output-file label_predictor_lstm/${RESULT_DIR}/predictions_dev.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${DEV_FILE_PATH}
allennlp predict --use-dataset-reader --silent --output-file label_predictor_lstm/${RESULT_DIR}/predictions_test.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${TEST_FILE_PATH}

### 4. Evaluate classifier

In [ ]:
def load_predictions(path):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            result.append(json.loads(line)["label"])
            
    print('length of result:', len(result))
    return result

In [ ]:
RESULT_DIR = 'results_30'

On dev set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(DEV_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
len(true)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
from utils.plot_confusion_matrix import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

labels = list(set(true))
labels.sort()
plot_confusion_matrix(confusion_matrix(true[:len(pred)], pred, labels), target_names=labels, normalize=True)

In [ ]:
class_mapper = {
    'background_NS': 'elaboration_NS',
    'background_SN': 'preparation_NS',
    'evaluation_SN': 'elaboration_NS',
    'evidence_NS': 'elaboration_NS',
}

true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]

print(classification_report(true[:len(pred)], pred))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

On test set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(TEST_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')

print(classification_report(true[:len(pred)], pred))

In [ ]:
len(true)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

Join some undistinguishable classes

In [ ]:
class_mapper = {
    'background_r': 'elaboration_r',
    'antithesis_r': 'elaboration_r'
}

true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]

print(classification_report(true[:len(pred)], pred))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))